# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [18]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name.rstrip(),[]))
col

[('Postcode', []), ('Borough', []), ('Neighbourhood', [])]

In [19]:
old0 = ""
old1 = ""
old2 = ""
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #Iterate through each element of the row
    i=0
    for t in T.iterchildren():
        if i == 0:
            item0 = t.text_content().rstrip()
        elif i == 1:
            item1 = t.text_content().rstrip()
        else:
            item2 = t.text_content().rstrip()
        i+=1
    
    #Memorize or append item to table
    if item1 != "Not assigned":
        if item2 == "Not assigned":
            item2 = item1
        if item0 != old0:
            # append previous row to table
            if old0 != "":
                col[0][1].append(old0)
                col[1][1].append(old1)
                col[2][1].append(old2)
            # replace old row with current item
            old0 = item0
            old1 = item1
            old2 = item2
        else:
            old2 = old2 + "," + item2

# append final row to table
if old0 != "":
    col[0][1].append(old0)
    col[1][1].append(old1)
    col[2][1].append(old2)

In [20]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [21]:
df.tail()

,Postcode,Borough,Neighbourhood
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


In [22]:
df.shape

(103, 3)